In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [2]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("강동구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5 
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Gangdong-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 청기와타운 길동사거리점
2. 동래복국가마솥추어탕
3. 강동일식
4. 소우주 천호본점
5. 시집그릴하우스
6. 하루노유키
7. 메종 드 플레터 스타시티서울점
8. 생어거스틴 성내점
9. 스시야미 천호본점
10. 로니로티 천호점
11. 등갈비달인 본점
12. 119닭발 천호 본점
13. 그루바
14. 향돈
15. 와고참숯갈비
16. 놉스 천호점
17. 긴자 올림픽점
18. 브랜뉴하이몬드
19. 굴한마당
20. 수지상회 천호점
21. 이한진숙성회
22. 미테이블 천호직영점
23. 쭈꾸미도사 천호본점
24. 동신상회 상일점
25. 육대쌈 암사점
26. 엄마손만두
27. 와카노키친
28. 경송 천호본점
29. 이대성의진면목 강동직영점
30. 고양꼬치 천호점
31. 풍년상회
32. 온도계
33. 푸지미곱창
34. 도리상회 천호본점
35. 차이나린찐
36. 온고식당
37. 동국성
38. 날로먹는수산시장 길동점
39. 압구정화로구이 강동역점
40. 다이닝갈비 강동역점
41. 미드데이썬
42. 바다회집
43. 쿠우쿠우 골드 강동점
44. 스윗솔트
45. 박대감황금돈가
46. 냅다청양집 강동구청점
47. 육해공
48. 블랑제리11-17
49. 다람
50. 화로상회 천호점
51. 꿀꿀진순대 강동본점
52. 돼봉삼겹살 천호점
53. 미테이블 올림픽공원직영점
54. 송하정스시
55. 우대포 고덕역점
56. 길동한우마을
57. 강동반상
58. 상무초밥 길동점
59. 멘야세븐
60. 버섯잔치집 본점
61. 브리에
62. 앙상블
63. 카페제이
64. 더식당
65. 꼼떼 둔촌본점
66. 진정한족발 본점
67. 미나리산장 천호점
68. 안녕식당
69. 이모네아구탕찜
70. 신사소곱창 천호점
71. 갓잇 올림픽공원점
72. 김장하는고기집향산리 암사역점
73. 스시현
74. YUP
75. 연우식당
76. 강경불고기 강동구청점
'다음' 버튼 클릭: 페이지 2로 이동 중
현재 2페이지) 총 76개의 가게를 찾았습니다.
77. 진미오뎅
78. 용